In [1]:
import sys
import numpy as np
sys.path.append("/home/kalic/Desktop/AI_P/CyberBattleSim")
from cyberbattle.simulation import model
from cyberbattle.samples.topology_nodeinfo import topology_and_nodeinfo_env
import gym
import cyberbattle.agents.baseline.agent_wrapper as w
import logging
import cyberbattle.agents.baseline.learner as learner
import cyberbattle.agents.baseline.plotting as p
import cyberbattle.agents.baseline.agent_randomcredlookup as rca
import cyberbattle.agents.baseline.agent_tabularqlearning as tqa
import cyberbattle.agents.baseline.agent_dql as dqla
from cyberbattle.agents.baseline.agent_wrapper import Verbosity

In [2]:
env_size = None
iteration_count = 300
training_episode_count = 10
eval_episode_count = 2
maximum_node_count = 30
maximum_total_credentials = 30

In [8]:
gymid = "topology_and_nodeinfo_env-v0"
# size_N, size_NGN, dis_type, network_cc, connect_percent
gym_env = gym.make(gymid, size_N = 7, size_NGN = 2, dis_type = "gradient_up", network_cc = "chained", connect_percent = 0.9)

7 d
{'data': NodeInfo(services=[ListeningService(name='HTTP', allowedCredentials=[], running=True), ListeningService(name='HTTPs', allowedCredentials=[], running=True)], vulnerabilities={'CVE-2020-3433': VulnerabilityInfo(description='Zenphoto, HTTP, HTTPs', type=<VulnerabilityType.LOCAL: 1>, outcome=<cyberbattle.simulation.model.AdminEscalation object at 0x7f9533dc8fd0>, precondition=<cyberbattle.simulation.model.Precondition object at 0x7f9533dc8a30>, rates=Rates(probingDetectionRate=0, exploitDetectionRate=0.2, successRate=1.0), URL='https://nvd.nist.gov/vuln/detail/CVE-2020-433', cost=1.0, reward_string=''), 'CVE-2020-3153': VulnerabilityInfo(description='Cisco AnyConnect <=4.5.02042', type=<VulnerabilityType.LOCAL: 1>, outcome=<cyberbattle.simulation.model.AdminEscalation object at 0x7f9533dc90a0>, precondition=<cyberbattle.simulation.model.Precondition object at 0x7f9533dc8f70>, rates=Rates(probingDetectionRate=0, exploitDetectionRate=0.2, successRate=1.0), URL='https://nvd.nist.

In [4]:
ep = w.EnvironmentBounds.of_identifiers(
    maximum_node_count=maximum_node_count,
    maximum_total_credentials=maximum_total_credentials,
    identifiers=gym_env.identifiers
)

In [9]:
# Evaluate the Deep Q-learning agent
my_dql_run1 = learner.epsilon_greedy_search(
    cyberbattle_gym_env=gym_env,
    environment_properties=ep,
    learner=dqla.DeepQLearnerPolicy(
        ep=ep,
        gamma=0.015,
        replay_memory_size=10000,
        target_update=10,
        batch_size=512,
        # torch default learning rate is 1e-2
        # a large value helps converge in less episodes
        learning_rate=0.01
    ),
    episode_count=training_episode_count,
    iteration_count=iteration_count,
    epsilon=0.90,
    epsilon_exponential_decay=5000,
    epsilon_minimum=0.10,
    verbosity=Verbosity.Quiet,
    render=False,
    plot_episodes_length=False,
    title="DQL"
)

###### DQL
Learning with: episode_count=10,iteration_count=300,ϵ=0.9,ϵ_min=0.1, ϵ_expdecay=5000,γ=0.015, lr=0.01, replaymemory=10000,
batch=512, target_update=10


  Episode 1 stopped at t=300 
  Episode 2 stopped at t=300 
  Episode 3 stopped at t=300 
  Episode 4 stopped at t=300 
  Episode 5 stopped at t=300 
  Episode 6 stopped at t=300 
  Episode 7 stopped at t=300 
  Episode 8 stopped at t=300 
  Episode 9 stopped at t=300 
  Episode 10 stopped at t=300 
simulation ended


In [10]:
print(my_dql_run1['all_steps'], my_dql_run1['all_episodes_rewards'])

[34, 247, 104, 61, 76, 74, 102, 32, 63, 40] [138.0, 138.0, 138.0, 138.0, 138.0, 138.0, 138.0, 138.0, 138.0, 138.0]


In [11]:
DQN_steps_N = []
DQN_rewards_N = []
my_steps_N = []
my_rewards_N = []

In [12]:
N = 50
for i in range(N):
    # Evaluate the Deep Q-learning agent
    gym_env = gym.make(gymid, size_N = 7, size_NGN = 2, dis_type = "gradient_up", network_cc = "chained", connect_percent = 0.9)

    my_dql_run = learner.epsilon_greedy_search(
        cyberbattle_gym_env=gym_env,
        environment_properties=ep,
        learner=my_dql_run1['learner'],
        episode_count=training_episode_count,
        iteration_count=iteration_count,
        epsilon=0.90,
        epsilon_exponential_decay=5000,
        epsilon_minimum=0.10,
        verbosity=Verbosity.Quiet,
        render=False,
        plot_episodes_length=False,
        title="DQL"
    )

    my_steps_N.append(my_dql_run['all_steps'])
    my_rewards_N.append(my_dql_run['all_episodes_rewards'])
    print(i, my_dql_run['all_steps'], my_dql_run['all_episodes_rewards'])
    my_dql_run1 = my_dql_run
    
    dql_run = learner.epsilon_greedy_search(
        cyberbattle_gym_env=gym_env,
        environment_properties=ep,
        learner=dqla.DeepQLearnerPolicy(
            ep=ep,
            gamma=0.015,
            replay_memory_size=10000,
            target_update=10,
            batch_size=512,
            # torch default learning rate is 1e-2
            # a large value helps converge in less episodes
            learning_rate=0.01
        ),
        episode_count=training_episode_count,
        iteration_count=iteration_count,
        epsilon=0.90,
        epsilon_exponential_decay=5000,
        epsilon_minimum=0.10,
        verbosity=Verbosity.Quiet,
        render=False,
        plot_episodes_length=False,
        title="DQL"
    )
    DQN_steps_N.append(dql_run['all_steps'])
    DQN_rewards_N.append(dql_run['all_episodes_rewards'])
    print(i, dql_run['all_steps'], dql_run['all_episodes_rewards'])
    


7 d
{'data': NodeInfo(services=[ListeningService(name='HTTP', allowedCredentials=[], running=True), ListeningService(name='HTTPs', allowedCredentials=[], running=True)], vulnerabilities={'CVE-2020-3433': VulnerabilityInfo(description='Zenphoto, HTTP, HTTPs', type=<VulnerabilityType.LOCAL: 1>, outcome=<cyberbattle.simulation.model.AdminEscalation object at 0x7f9533dc8fd0>, precondition=<cyberbattle.simulation.model.Precondition object at 0x7f9533dc8a30>, rates=Rates(probingDetectionRate=0, exploitDetectionRate=0.2, successRate=1.0), URL='https://nvd.nist.gov/vuln/detail/CVE-2020-433', cost=1.0, reward_string=''), 'CVE-2020-3153': VulnerabilityInfo(description='Cisco AnyConnect <=4.5.02042', type=<VulnerabilityType.LOCAL: 1>, outcome=<cyberbattle.simulation.model.AdminEscalation object at 0x7f9533dc90a0>, precondition=<cyberbattle.simulation.model.Precondition object at 0x7f9533dc8f70>, rates=Rates(probingDetectionRate=0, exploitDetectionRate=0.2, successRate=1.0), URL='https://nvd.nist.

  Episode 1 stopped at t=300 
  Episode 2 stopped at t=300 
  Episode 3 stopped at t=300 
  Episode 4 stopped at t=300 
  Episode 5 stopped at t=300 
  Episode 6 stopped at t=300 
  Episode 7 stopped at t=300 
  Episode 8 stopped at t=300 
  Episode 9 stopped at t=300 
  Episode 10 stopped at t=300 
simulation ended
0 [10, 1, 1, 5, 6, 2, 33, 4, 2, 1] [16.0, 16.0, 16.0, 16.0, 16.0, 16.0, 16.0, 16.0, 16.0, 16.0]
###### DQL
Learning with: episode_count=10,iteration_count=300,ϵ=0.9,ϵ_min=0.1, ϵ_expdecay=5000,γ=0.015, lr=0.01, replaymemory=10000,
batch=512, target_update=10
  Episode 1 stopped at t=300 
  Episode 2 stopped at t=300 
  Episode 3 stopped at t=300 
  Episode 4 stopped at t=300 
  Episode 5 stopped at t=300 
  Episode 6 stopped at t=300 
  Episode 7 stopped at t=300 
  Episode 8 stopped at t=300 
  Episode 9 stopped at t=300 
  Episode 10 stopped at t=300 
simulation ended
0 [4, 6, 3, 9, 4, 4, 3, 4, 4, 5] [16.0, 16.0, 16.0, 16.0, 16.0, 16.0, 16.0, 16.0, 16.0, 16.0]
7 d
{'data':

In [9]:
for i in range(10):
    #print(my_steps_N[i],DQN_steps_N[i])
    print(my_rewards_N[i], DQN_rewards_N[i])
    sii = 0
    for j in range(10):
        sii += my_steps_N[i][j] - DQN_steps_N[i][j]
    print(sii)


[207.0, 207.0, 207.0, 207.0, 207.0, 207.0, 207.0, 207.0, 207.0, 207.0] [207.0, 207.0, 207.0, 207.0, 207.0, 207.0, 207.0, 207.0, 207.0, 207.0]
-382
[274.0, 274.0, 274.0, 274.0, 274.0, 274.0, 274.0, 274.0, 274.0, 274.0] [119.0, 274.0, 274.0, 274.0, 274.0, 274.0, 274.0, 274.0, 274.0, 274.0]
53
[315.0, 315.0, 315.0, 315.0, 315.0, 315.0, 315.0, 315.0, 315.0, 315.0] [315.0, 315.0, 315.0, 315.0, 315.0, 315.0, 315.0, 315.0, 315.0, 315.0]
-519
[472.0, 472.0, 472.0, 472.0, 472.0, 472.0, 472.0, 472.0, 472.0, 472.0] [472.0, 472.0, 472.0, 472.0, 472.0, 472.0, 472.0, 472.0, 472.0, 472.0]
57
[363.0, 363.0, 363.0, 363.0, 363.0, 363.0, 363.0, 363.0, 363.0, 363.0] [357.0, 363.0, 363.0, 363.0, 363.0, 363.0, 363.0, 363.0, 363.0, 363.0]
37
[80.0, 80.0, 80.0, 80.0, 80.0, 80.0, 80.0, 80.0, 80.0, 80.0] [80.0, 80.0, 80.0, 80.0, 80.0, 80.0, 80.0, 80.0, 80.0, 80.0]
-245
[311.0, 311.0, 311.0, 311.0, 311.0, 311.0, 311.0, 311.0, 311.0, 311.0] [311.0, 311.0, 311.0, 311.0, 311.0, 311.0, 311.0, 311.0, 311.0, 311.0]
36

In [12]:
N = 40
for i in range(N):
    # Evaluate the Deep Q-learning agent
    gym_env = gym.make(gymid, size_N = 7, size_NGN = 2, dis_type = "gradient_up", network_cc = "chained", connect_percent = 0.9)
    my_dql_run = learner.epsilon_greedy_search(
        cyberbattle_gym_env=gym_env,
        environment_properties=ep,
        learner=my_dql_run1['learner'],
        episode_count=training_episode_count,
        iteration_count=iteration_count,
        epsilon=0.90,
        epsilon_exponential_decay=5000,
        epsilon_minimum=0.10,
        verbosity=Verbosity.Quiet,
        render=False,
        plot_episodes_length=False,
        title="DQL"
    )

    my_steps_N.append(my_dql_run['all_steps'])
    my_rewards_N.append(my_dql_run['all_episodes_rewards'])
    print(i, my_dql_run['all_steps'], my_dql_run['all_episodes_rewards'])
    my_dql_run1 = my_dql_run
    
    dql_run = learner.epsilon_greedy_search(
        cyberbattle_gym_env=gym_env,
        environment_properties=ep,
        learner=dqla.DeepQLearnerPolicy(
            ep=ep,
            gamma=0.015,
            replay_memory_size=10000,
            target_update=10,
            batch_size=512,
            # torch default learning rate is 1e-2
            # a large value helps converge in less episodes
            learning_rate=0.01
        ),
        episode_count=training_episode_count,
        iteration_count=iteration_count,
        epsilon=0.90,
        epsilon_exponential_decay=5000,
        epsilon_minimum=0.10,
        verbosity=Verbosity.Quiet,
        render=False,
        plot_episodes_length=False,
        title="DQL"
    )
    DQN_steps_N.append(dql_run['all_steps'])
    DQN_rewards_N.append(dql_run['all_episodes_rewards'])
    print(i, dql_run['all_steps'], dql_run['all_episodes_rewards'])

7 d
{'data': NodeInfo(services=[ListeningService(name='SSH', allowedCredentials=[], running=True)], vulnerabilities={'CVE-2021-4034': VulnerabilityInfo(description='Polkit', type=<VulnerabilityType.LOCAL: 1>, outcome=<cyberbattle.simulation.model.AdminEscalation object at 0x7f2390336a90>, precondition=<cyberbattle.simulation.model.Precondition object at 0x7f23903368b0>, rates=Rates(probingDetectionRate=0, exploitDetectionRate=0.2, successRate=1.0), URL='https://nvd.nist.gov/vuln/detail/cve-2021-4034', cost=1.0, reward_string=''), 'SMBBF': VulnerabilityInfo(description='SSH Brute Force', type=<VulnerabilityType.REMOTE: 2>, outcome=<cyberbattle.simulation.model.LateralMove object at 0x7f2390336880>, precondition=<cyberbattle.simulation.model.Precondition object at 0x7f23903366a0>, rates=Rates(probingDetectionRate=0, exploitDetectionRate=0.2, successRate=1.0), URL='https://attack.mitre.org/techniques/T1110/', cost=1.0, reward_string=''), 'SSHBF': VulnerabilityInfo(description='SSH Brute F

  Episode 1 stopped at t=300 
  Episode 2 stopped at t=300 
  Episode 3 stopped at t=300 
  Episode 4 stopped at t=300 
  Episode 5 stopped at t=300 
  Episode 6 stopped at t=300 
  Episode 7 stopped at t=300 
  Episode 8 stopped at t=300 
  Episode 9 stopped at t=300 
  Episode 10 stopped at t=300 
simulation ended
0 [55, 43, 39, 99, 195, 89, 101, 58, 44, 114] [314.0, 314.0, 314.0, 314.0, 314.0, 314.0, 314.0, 314.0, 314.0, 314.0]
###### DQL
Learning with: episode_count=10,iteration_count=300,ϵ=0.9,ϵ_min=0.1, ϵ_expdecay=5000,γ=0.015, lr=0.01, replaymemory=10000,
batch=512, target_update=10
  Episode 1 stopped at t=300 
  Episode 2 stopped at t=300 
  Episode 3 stopped at t=300 
  Episode 4 stopped at t=300 
  Episode 5 stopped at t=300 
  Episode 6 stopped at t=300 
  Episode 7 stopped at t=300 
  Episode 8 stopped at t=300 
  Episode 9 stopped at t=300 
  Episode 10 stopped at t=300 
simulation ended
0 [147, 207, 189, 81, 43, 51, 34, 76, 79, 103] [314.0, 314.0, 314.0, 314.0, 314.0, 31

In [32]:
# import matplotlib
# matplotlib.use("TkAgg")
import matplotlib.pyplot as plt

In [13]:
x = list(range(50))
si = []
for i in range(50):
    #print(my_steps_N[i],DQN_steps_N[i])
    # print(my_rewards_N[i], DQN_rewards_N[i])
    sii = 0
    for j in range(10):
        sii += my_steps_N[i][j] - DQN_steps_N[i][j]
    si.append(sii)
print(si)


[19, 35, 229, 107, -403, -93, -290, -56, -195, 282, 0, -137, -167, 267, -70, -206, -48, -298, -163, 161, -110, -91, -160, 3, -24, 105, 83, -76, -195, -219, -82, -80, -342, -200, -12, -120, -168, -63, 176, -27, -16, -185, 163, -268, -4, 14, 203, -303, -24, -157]


In [18]:
np.savetxt('my_steps_N.txt', my_steps_N, fmt='%d', delimiter=",")
np.savetxt('DQN_steps_N.txt', DQN_steps_N, fmt='%d', delimiter=",")
np.savetxt('my_rewards_N.txt', my_rewards_N, fmt='%d', delimiter=",")
np.savetxt('DQN_rewards_N.txt', DQN_rewards_N, fmt='%d', delimiter=",")

In [19]:
import pickle

In [20]:
with open('qianyitest100.pkl','wb') as file:
    pickle.dump(my_dql_run, file)

In [21]:
with open('qianyitest100.pkl','rb') as file:
    load_m = pickle.load(file)

In [22]:
ceshi_dql_run = learner.epsilon_greedy_search(
    cyberbattle_gym_env=gym_env,
    environment_properties=ep,
    learner=load_m['learner'],
    episode_count=training_episode_count,
    iteration_count=iteration_count,
    epsilon=0.90,
    epsilon_exponential_decay=5000,
    epsilon_minimum=0.10,
    verbosity=Verbosity.Quiet,
    render=False,
    plot_episodes_length=False,
    title="DQL"
)

###### DQL
Learning with: episode_count=10,iteration_count=300,ϵ=0.9,ϵ_min=0.1, ϵ_expdecay=5000,γ=0.015, lr=0.01, replaymemory=10000,
batch=512, target_update=10
  Episode 1 stopped at t=300 
  Episode 2 stopped at t=300 
  Episode 3 stopped at t=300 
  Episode 4 stopped at t=300 
  Episode 5 stopped at t=300 
  Episode 6 stopped at t=300 
  Episode 7 stopped at t=300 
  Episode 8 stopped at t=300 
  Episode 9 stopped at t=300 
  Episode 10 stopped at t=300 
simulation ended


In [34]:
N = 40
for i in range(N):
    # Evaluate the Deep Q-learning agent
    gym_env = gym.make(gymid, size_N = 7, size_NGN = 2, dis_type = "gradient_up", network_cc = "chained", connect_percent = 0.9)
    my_dql_run = learner.epsilon_greedy_search(
        cyberbattle_gym_env=gym_env,
        environment_properties=ep,
        learner=my_dql_run1['learner'],
        episode_count=training_episode_count,
        iteration_count=iteration_count,
        epsilon=0.90,
        epsilon_exponential_decay=5000,
        epsilon_minimum=0.10,
        verbosity=Verbosity.Quiet,
        render=False,
        plot_episodes_length=False,
        title="DQL"
    )

    my_steps_N.append(my_dql_run['all_steps'])
    my_rewards_N.append(my_dql_run['all_episodes_rewards'])
    print(i, my_dql_run['all_steps'], my_dql_run['all_episodes_rewards'])
    my_dql_run1 = my_dql_run
    
    dql_run = learner.epsilon_greedy_search(
        cyberbattle_gym_env=gym_env,
        environment_properties=ep,
        learner=dqla.DeepQLearnerPolicy(
            ep=ep,
            gamma=0.015,
            replay_memory_size=10000,
            target_update=10,
            batch_size=512,
            # torch default learning rate is 1e-2
            # a large value helps converge in less episodes
            learning_rate=0.01
        ),
        episode_count=training_episode_count,
        iteration_count=iteration_count,
        epsilon=0.90,
        epsilon_exponential_decay=5000,
        epsilon_minimum=0.10,
        verbosity=Verbosity.Quiet,
        render=False,
        plot_episodes_length=False,
        title="DQL"
    )
    DQN_steps_N.append(dql_run['all_steps'])
    DQN_rewards_N.append(dql_run['all_episodes_rewards'])
    print(i, dql_run['all_steps'], dql_run['all_episodes_rewards'])

7 d
why
7 s
###### DQL
Learning with: episode_count=10,iteration_count=300,ϵ=0.9,ϵ_min=0.1, ϵ_expdecay=5000,γ=0.015, lr=0.01, replaymemory=10000,
batch=512, target_update=10
  Episode 1 stopped at t=300 
  Episode 2 stopped at t=300 
  Episode 3 stopped at t=300 
  Episode 4 stopped at t=300 
  Episode 5 stopped at t=300 
  Episode 6 stopped at t=300 
  Episode 7 stopped at t=300 
  Episode 8 stopped at t=300 
  Episode 9 stopped at t=300 
  Episode 10 stopped at t=300 
simulation ended
0 [1, 2, 6, 8, 6, 1, 2, 12, 1, 1] [11.0, 11.0, 11.0, 11.0, 11.0, 11.0, 11.0, 11.0, 11.0, 11.0]
###### DQL
Learning with: episode_count=10,iteration_count=300,ϵ=0.9,ϵ_min=0.1, ϵ_expdecay=5000,γ=0.015, lr=0.01, replaymemory=10000,
batch=512, target_update=10
  Episode 1 stopped at t=300 
  Episode 2 stopped at t=300 
  Episode 3 stopped at t=300 
  Episode 4 stopped at t=300 
  Episode 5 stopped at t=300 
  Episode 6 stopped at t=300 
  Episode 7 stopped at t=300 
  Episode 8 stopped at t=300 
  Episode 9

In [30]:
plt.plot(np.arange(1,61,1),si,"ob:")
plt.show()

/tmp/ipykernel_8940/308180493.py:2: UserWarning: Matplotlib is currently using agg, which is a non-GUI backend, so cannot show the figure.
  plt.show()


In [ ]:
# Evaluate the Deep Q-learning agent
dql_run = learner.epsilon_greedy_search(
    cyberbattle_gym_env=gym_env,
    environment_properties=ep,
    learner=dqla.DeepQLearnerPolicy(
        ep=ep,
        gamma=0.015,
        replay_memory_size=10000,
        target_update=10,
        batch_size=512,
        # torch default learning rate is 1e-2
        # a large value helps converge in less episodes
        learning_rate=0.01
    ),
    episode_count=training_episode_count,
    iteration_count=iteration_count,
    epsilon=0.90,
    epsilon_exponential_decay=5000,
    epsilon_minimum=0.10,
    verbosity=Verbosity.Quiet,
    render=False,
    plot_episodes_length=False,
    title="DQL"
)

###### DQL
Learning with: episode_count=10,iteration_count=300,ϵ=0.9,ϵ_min=0.1, ϵ_expdecay=5000,γ=0.015, lr=0.01, replaymemory=10000,
batch=512, target_update=10


/home/kalic/Desktop/AI_P/CyberBattleSim/cyberbattle/agents/baseline/agent_dql.py:382: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:230.)
  state_batch = torch.tensor(states_to_consider).to(device)


  Episode 1 stopped at t=300 
  Breakdown [Reward/NoReward (Success rate)]
    explore-local: 0/83 (0.00)
    explore-remote: 5/207 (0.02)
    explore-connect: 0/0 (NaN)
    exploit-local: 0/0 (NaN)
    exploit-remote: 1/4 (0.20)
    exploit-connect: 0/0 (NaN)
  exploit deflected to exploration: 32
  Episode 2 stopped at t=300 
  Breakdown [Reward/NoReward (Success rate)]
    explore-local: 0/86 (0.00)
    explore-remote: 6/166 (0.03)
    explore-connect: 0/0 (NaN)
    exploit-local: 0/0 (NaN)
    exploit-remote: 0/42 (0.00)
    exploit-connect: 0/0 (NaN)
  exploit deflected to exploration: 11
  Episode 3 stopped at t=300 
  Breakdown [Reward/NoReward (Success rate)]
    explore-local: 0/77 (0.00)
    explore-remote: 5/159 (0.03)
    explore-connect: 0/0 (NaN)
    exploit-local: 0/0 (NaN)
    exploit-remote: 1/58 (0.02)
    exploit-connect: 0/0 (NaN)
  exploit deflected to exploration: 4
  Episode 4 stopped at t=300 
  Breakdown [Reward/NoReward (Success rate)]
    explore-local: 0/64 

In [ ]:
print(dql_run['all_steps'],  dql_run['all_episodes_rewards'])
print(dql_run['all_episodes_availability'],dql_run['learner'])

[105, 73, 53, 32, 51, 77, 32, 53, 24, 23] [162.0, 162.0, 162.0, 162.0, 162.0, 162.0, 162.0, 162.0, 162.0, 162.0]
[[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 

In [ ]:
gymid = "topology_and_nodeinfo_env-v0"
# size_N, size_NGN, dis_type, network_cc, connect_percent
gym_env2 = gym.make(gymid, size_N = 7, size_NGN = 2, dis_type = "gradient_up", network_cc = "chained", connect_percent = 0.9)

7 d
why
7 s


In [ ]:
# Evaluate the Deep Q-learning agent
dql_run2 = learner.epsilon_greedy_search(
    cyberbattle_gym_env=gym_env,
    environment_properties=ep,
    learner=dql_run['learner'](
        ep=ep,
        gamma=0.015,
        replay_memory_size=10000,
        target_update=10,
        batch_size=512,
        # torch default learning rate is 1e-2
        # a large value helps converge in less episodes
        learning_rate=0.01
    ),
    episode_count=training_episode_count,
    iteration_count=iteration_count,
    epsilon=0.90,
    epsilon_exponential_decay=5000,
    epsilon_minimum=0.10,
    verbosity=Verbosity.Quiet,
    render=False,
    plot_episodes_length=False,
    title="DQL"
)

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
x = range(training_episode_count)
steps = dql_run['all_steps']
y = np.mean(steps,axis=0); ystd = np.std(steps,axis=0)
plt.plot(x,y,label='p=0.1')
plt.fill_between(x,y-ystd,y+ystd,alpha=.5)